In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow
from tensorflow.keras.layers import Embedding, Dense,Bidirectional,LSTM
from tensorflow.keras.models import Sequential

In [2]:
df = pd.read_csv('/content/dialogs.txt',sep = "|", names=['main','dis'])

In [3]:
df.head()

,main,dis
0,"hi, how are you doing?",i'm fine. how about yourself?
1,i'm fine. how about yourself?,i'm pretty good. thanks for asking.
2,i'm pretty good. thanks for asking.,no problem. so how have you been?
3,no problem. so how have you been?,i've been great. what about you?
4,i've been great. what about you?,i've been good. i'm in school right now.


In [4]:
df.drop('dis',axis=1, inplace=True)

In [5]:
data = np.array(df.main)

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index)+1


In [7]:
input_sequences = []
for line in data:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tensorflow.keras.utils.to_categorical(labels, num_classes=total_words)

In [8]:
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    Bidirectional(LSTM(150)),
    Dense(total_words, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=100, verbose=1)
#print model.summary()
print(model)

Epoch 1/100
625/625 [==============================] - 13s 9ms/step - loss: 6.3004 - accuracy: 0.0455
Epoch 2/100
625/625 [==============================] - 5s 9ms/step - loss: 5.5937 - accuracy: 0.0868
Epoch 3/100
625/625 [==============================] - 6s 9ms/step - loss: 5.1498 - accuracy: 0.1183
Epoch 4/100
625/625 [==============================] - 6s 9ms/step - loss: 4.7777 - accuracy: 0.1495
Epoch 5/100
625/625 [==============================] - 5s 9ms/step - loss: 4.4502 - accuracy: 0.1717
Epoch 6/100
625/625 [==============================] - 6s 9ms/step - loss: 4.1594 - accuracy: 0.1958
Epoch 7/100
625/625 [==============================] - 6s 9ms/step - loss: 3.8942 - accuracy: 0.2178
Epoch 8/100
625/625 [==============================] - 6s 9ms/step - loss: 3.6506 - accuracy: 0.2459
Epoch 9/100
625/625 [==============================] - 6s 9ms/step - loss: 3.4171 - accuracy: 0.2761
Epoch 10/100
625/625 [==============================] - 6s 9ms/step - loss: 3.1985 - accur

In [12]:
seed_text = "hello"
next_words = 3
  
for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)

1/1 [==============================] - 0s 16ms/step
hello may i speak


In [14]:
model.save('word_pred.h5')